In [1]:
import operator as op
import src.ea.ea_lib as ea
import src.ea.problem_data as pb
import src.ea.utilities as ut
import random as rd
import time
import numpy as np

In [2]:
#parameters
rd.seed(1)
population_size = 10
problem_name = "spect"
levels_back = 2
n_rows = 4
n_columns = 9
allow_input_to_output = False
inputs_available_to_all_columns = False
functions = [op.add,op.sub,op.mul,ut.safe_divide_one]
objectives = [
    ea.Objective(name="acc0", to_max = True, best=1, worst=0),
    ea.Objective(name="acc1", to_max = True, best=1, worst=0)
    ]

In [3]:
#Definition
dataset = pb.Dataset()
dataset.load_problem(name = problem_name)
cgp = ea.CGP_Representation(
            dataset.x_train.shape[1]
            ,1
            ,levels_back
            ,n_rows 
            ,n_columns
            ,allow_input_to_output
            ,inputs_available_to_all_columns
            ,*functions)

In [4]:
#Execution
start_t = time.time()

generation = 0
#Random initial population. Specific initial conditions for the population can be specified here
graphs = [cgp.create_random(seed = rd.random()) for _ in range(population_size)]
#create instances of Individual to be grouped in the population
population = [ea.Individual(r,generation) for r in graphs]
for ind in population:
    outputs = {}
    for i,data_row in enumerate(dataset.x_train):
        output_dict = ind.representation.evaluate(data_row = data_row)
        #Extracting the 0th index output gene
        output = output_dict[0]
        #The raw output needs to be transformed, in this case it is simply rounded to the closest int
        output_rounded = ut.custom_round(output)
        outputs[i] = output_rounded
    #Each objective has its own evaluation method
    acc0 = ut.accuracy_in_label(y=dataset.y_train, y_output=outputs, label = 0)
    acc1 = ut.accuracy_in_label(y=dataset.y_train, y_output=outputs, label = 1)
    ind.update_evaluation(objective=objectives[0],value=acc0)
    ind.update_evaluation(objective=objectives[1],value=acc1)
    ind.update_semantics_all(semantics_all = outputs)

print("Execution time: ",str(time.time()-start_t))

evals0 = []
evals1 = []
actives = []
for i,ind in enumerate(population):
    evals0.append(ind.evaluations[objectives[0].name])
    evals1.append(ind.evaluations[objectives[1].name])
    actives.append(len(ind.representation.active_genotype))
print("Mean acc0", np.mean(evals0))
print("Mean acc1", np.mean(evals1))
print("Mean active nodes", np.mean(actives))

Execution time:  0.04590463638305664
Mean acc0 0.31348314606741573
Mean acc1 0.0951310861423221
Mean active nodes 6.0


In [7]:
for ind in population:
    print(ind.evaluations)


{'acc0': 0.5880149812734082, 'acc1': 0.0}
{'acc0': 0.1198501872659176, 'acc1': 0.3408239700374532}
{'acc0': 0.26217228464419473, 'acc1': 0.0}
{'acc0': 0.5280898876404494, 'acc1': 0.09363295880149813}
{'acc0': 0.1647940074906367, 'acc1': 0.19850187265917604}
{'acc0': 0.5280898876404494, 'acc1': 0.018726591760299626}
{'acc0': 0.0, 'acc1': 0.2808988764044944}
{'acc0': 0.36329588014981273, 'acc1': 0.0}
{'acc0': 0.08239700374531835, 'acc1': 0.0}
{'acc0': 0.49812734082397003, 'acc1': 0.018726591760299626}


In [6]:
sorted_pop = ea.sort_population(population,objectives)
for ind in sorted_pop:
    print(ind.evaluations)

{'acc0': 0.5880149812734082, 'acc1': 0.0}
{'acc0': 0.5280898876404494, 'acc1': 0.09363295880149813}
{'acc0': 0.5280898876404494, 'acc1': 0.018726591760299626}
{'acc0': 0.49812734082397003, 'acc1': 0.018726591760299626}
{'acc0': 0.36329588014981273, 'acc1': 0.0}
{'acc0': 0.26217228464419473, 'acc1': 0.0}
{'acc0': 0.1647940074906367, 'acc1': 0.19850187265917604}
{'acc0': 0.1198501872659176, 'acc1': 0.3408239700374532}
{'acc0': 0.08239700374531835, 'acc1': 0.0}
{'acc0': 0.0, 'acc1': 0.2808988764044944}


In [43]:
tournament_size = 4
population_size = len(population)
competitors = rd.sample(population,k=tournament_size)
winner = ea.sort_population(population=competitors, objectives = objectives)[0]
print(winner.evaluations)
    

{'acc0': 0.5280898876404494, 'acc1': 0.09363295880149813}
